In [ ]:
"""
Idea Assistant Notebook
-----------------------
This notebook builds an interactive assistant to:
1. Extract project guide content from a PDF.
2. Convert the extracted text into Markdown format.
3. Use OpenAI’s API to generate step-by-step project guides based on user input.
4. Provide a simple web-based interface using Gradio.

Sections:
- Imports
- PDF Utilities
- OpenAI Setup
- Project Guide Generation
- Gradio UI
"""

# ==============================
# Imports
# ==============================
import fitz  # PyMuPDF for PDF text extraction
from IPython.display import Markdown, display
from openai import OpenAI
from dotenv import load_dotenv
import os
import openai
import gradio as gr


# ==============================
# PDF Utilities
# ==============================
def pdf_to_markdown(pdf_path: str) -> str:
    """
    Extracts text from a PDF and converts it into a Markdown-style paragraph.

    Args:
        pdf_path (str): Path to the input PDF file.

    Returns:
        str: Markdown-formatted string version of the PDF content.
    """
    doc = fitz.open(pdf_path)
    text = ""

    # Extract text page by page
    for page in doc:
        text += page.get_text("text") + "\n"

    # Clean formatting -> convert into markdown sections
    lines = [line.strip() for line in text.split("\n") if line.strip()]

    md_output = "# AI Project Guide\n\n"

    for line in lines:
        # Detect section headers
        if line in ["Overview", "Goals", "Implementation", "Deliverables", "Success Criteria"]:
            md_output += f"## {line}\n"
        else:
            # Treat as bullet point
            md_output += f"- {line}\n"

    # Save raw extracted text (not markdown) into a .md file for reference
    with open("Data/project_guide.md", "w") as f:
        f.write(text)

    return md_output.strip()


# ==============================
# OpenAI Setup
# ==============================
# Load environment variables (API keys from .env file)
load_dotenv()

# Connect to OpenAI API
client = OpenAI(api_key=os.getenv("BOBAD_OPENAI_KEY"))

# (Note: legacy `openai` library import also exists in this notebook)
openai.api_key = "BOBAD_OPENAI_KEY"  # Placeholder for actual .env usage


# ==============================
# Project Guide Generation
# ==============================
def get_project_guide(idea: str, experience: str, tools: str, guide_text: str) -> str:
    """
    Generates a tailored project guide using OpenAI’s GPT models.

    Args:
        idea (str): User-provided project idea.
        experience (str): Experience level (Beginner, Intermediate, Advanced).
        tools (str): Preferred tools, technologies, or frameworks.
        guide_text (str): Markdown guide text extracted from PDF (context).

    Returns:
        str: AI-generated step-by-step project guide in markdown format.
    """
    prompt = f"""
You are an AI assistant that writes step-by-step project guides.
Use the structure below:

{guide_text}

Project Idea: {idea}
Experience Level: {experience}
Preferred Tools: {tools}

Write a markdown guide with a motivating intro and clear steps.
"""

    response = client.chat.completions.create(
        model="gpt-5-mini",
        messages=[{"role": "system", "content": prompt}]
    )

    return response.choices[0].message.content


# ==============================
# Wrapper for Gradio Interface
# ==============================
# Load guide text (reference from saved file)
with open("Data/project_guide.md", "r") as f:
    guide_text = f.read()


def ai_idea_assistant(idea: str, experience: str, tools: str) -> str:
    """
    Wrapper function for Gradio UI.
    Passes user inputs to the project guide generator.

    Args:
        idea (str): Project idea.
        experience (str): User’s experience level.
        tools (str): Preferred tools/technologies.

    Returns:
        str: AI-generated markdown project guide.
    """
    return get_project_guide(idea, experience, tools, guide_text)


# ==============================
# Gradio UI
# ==============================
iface = gr.Interface(
    fn=ai_idea_assistant,
    inputs=[
        gr.Textbox(label="Project Idea"),
        gr.Dropdown(["Beginner", "Intermediate", "Advanced"], label="Experience Level"),
        gr.Textbox(label="Preferred Tools (comma-separated, optional)")
    ],
    outputs="markdown",
    title="AI Project Idea Assistant",
    description="Describe your project idea and get a tailored, step-by-step project guide!"
)

# Launch the UI (will open in a browser window)
iface.launch()
